In [2]:
import os
import pandas as pd
import numpy as np
import pathlib
import xarray as xr
import rioxarray
from typing import List
import arrow

In [3]:
read_path:str=r'D:\01Proj\Station-Montior-Sys\_data\perclock'
read_file:str='WL1217_DAT.08442'
full_path:str=str(pathlib.Path(read_path, read_file))
full_path

'D:\\01Proj\\Station-Montior-Sys\\_data\\perclock\\WL1217_DAT.08442'

#### 1- 读取整点风要素并生成对应集合

In [4]:
with open(full_path,'rb') as f:
    data=pd.read_csv(f, encoding='gbk', sep='\s+', header=None,
                                                infer_datetime_format=False)

In [5]:
data

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,20231217,519,587,607,582,518,420,315,225,151,...,608,148,115,906,619,1517,248,2140,9999,9999


In [6]:
data.shape

(1, 35)

In [7]:
rows=data.shape[0]
rows

1

#### 1-1 读取第一行  
对于风要素，line0 行  
index=0 时间 eg:20231217  
index=1 -> index=48 24组数  
风向 | 风速

In [8]:
data.iloc[0][:5]

0    20231217
1         519
2         587
3         607
4         582
Name: 0, dtype: int64

In [15]:
data.iloc[0][0]

20231217.0

In [14]:
data.iloc[0][1:25]

1     519
2     587
3     607
4     582
5     518
6     420
7     315
8     225
9     151
10    116
11    150
12    251
13    363
14    472
15    564
16    617
17    597
18    525
19    439
20    365
21    301
22    258
23    253
24    313
Name: 0, dtype: int64

In [20]:
range(24)[0]

0

In [24]:
type(data.iloc[0])

pandas.core.series.Series

In [15]:
series_surge=data.iloc[0][1:25]

In [16]:
int(data.iloc[0][0])

20231217

In [17]:
series_surge

1     519
2     587
3     607
4     582
5     518
6     420
7     315
8     225
9     151
10    116
11    150
12    251
13    363
14    472
15    564
16    617
17    597
18    525
19    439
20    365
21    301
22    258
23    253
24    313
Name: 0, dtype: int64

In [18]:
list_surge:List[dict]=[]
for index in range(24):
    temp_surge=series_surge[index+1]
    temp_dict={'index':index,'surge':temp_surge}
    list_surge.append(temp_dict)

In [20]:
list_surge

[{'index': 0, 'surge': 519},
 {'index': 1, 'surge': 587},
 {'index': 2, 'surge': 607},
 {'index': 3, 'surge': 582},
 {'index': 4, 'surge': 518},
 {'index': 5, 'surge': 420},
 {'index': 6, 'surge': 315},
 {'index': 7, 'surge': 225},
 {'index': 8, 'surge': 151},
 {'index': 9, 'surge': 116},
 {'index': 10, 'surge': 150},
 {'index': 11, 'surge': 251},
 {'index': 12, 'surge': 363},
 {'index': 13, 'surge': 472},
 {'index': 14, 'surge': 564},
 {'index': 15, 'surge': 617},
 {'index': 16, 'surge': 597},
 {'index': 17, 'surge': 525},
 {'index': 18, 'surge': 439},
 {'index': 19, 'surge': 365},
 {'index': 20, 'surge': 301},
 {'index': 21, 'surge': 258},
 {'index': 22, 'surge': 253},
 {'index': 23, 'surge': 313}]

In [1]:
from typing import List

In [7]:
# 读取整点数据(风场为例)
def read_wind_file(full_path:str)->List[dict]:
    """
        读取指定路径的风要素文件
    """
    list_wind:List[dict]=[]
    # step-1: 判断指定文件是否存在
    if pathlib.Path(full_path).exists():
        # step-2: 以 gbk 格式打开指定文件
        with open(full_path,'rb') as f:
            data=pd.read_csv(f, encoding='gbk', sep='\s+', header=None,
                                                infer_datetime_format=False)
            # step-3:获取文件的shape
            shape=data.shape
            # 总行数
            rows=data.shape[0]
            # 总列数
            columns=data.shape[1]
            if rows>0:                
                # 日期
                # eg: 20231217
                #     yyyy-mm-dd                
                dt_str:str=str(int(data.iloc[0][0]))
                # 设置起始时间(utc)
                # xxxx 12:00(utc)
                dt_start_utc:arrow.Arrow=arrow.Arrow(dt_str,'yyyymmdd').add(hour=-12)
                # 站点起始时间为昨天的20点(local)
                realdata_series:pd.Series=data.iloc[0][1:]
                
                step=2
                for index in range(24):
                    # print(index)
                    temp_dt_arrow:arrow.Arrow=dt_start_utc.add(hour=1)  
                    temp_ts:int=temp_dt_arrow.timestamp()               
                    temp_wd=series_wind[index*step+1]
                    temp_ws=series_wind[index*step+2]
                    temp_dict={'index':index,'ts':temp_ts,'wd':temp_wd, 'ws':temp_ws}
                    list_wind.append(temp_dict)
    return list_wind

In [22]:
# 获取极值
data.iloc[0][25:]

25     608
26     148
27     115
28     906
29     619
30    1517
31     248
32    2140
33    9999
34    9999
Name: 0, dtype: int64

In [24]:
data.iloc[0][25:].count()

10

In [26]:
for index,val in enumerate(data.iloc[0][25:]):
    print(val)

608
148
115
906
619
1517
248
2140
9999
9999


In [27]:
for index,val in enumerate(data.iloc[0][25:]):
    print(index)

0
1
2
3
4
5
6
7
8
9


In [28]:
2%2

0

In [29]:
1%2

1

In [30]:
3%2

1

In [31]:
5%2

1

In [32]:
6%2

0

In [33]:
0%2

0

In [35]:
'608'.rjust(4,'0')

'0608'

In [38]:
data.iloc[0][34]!=9999

False